In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
import re
import collections
import random
import math

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
pd.set_option('display.max_column', None)
train_input = pd.read_csv('train.csv')
test_input = pd.read_csv('test.csv')
print("train_input shape is: ", np.shape(train_input))
print("test_input shape is: ", np.shape(test_input))

train_input shape is:  (4743, 2)
test_input shape is:  (1701, 2)


In [4]:
# train_input['tweet'][:5]
for text in train_input['tweet'][:10]:
    print(text)
#     for word in text:
#         print(word)
    

The question in this election: Who can put the plans into action that will make your life better? https://t.co/XreEY9OicG
Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF
If we stand together, there's nothing we can't do. 

Make sure you're ready to vote: https://t.co/tTgeqxNqYm https://t.co/Q3Ymbb7UNy
Both candidates were asked about how they'd confront racial injustice. Only one had a real answer. https://t.co/sjnEokckis
Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc
When Donald Trump goes low...register to vote: https://t.co/tTgeqxNqYm https://t.co/DXz9dEwsZS
3) Has Trump offered a single proposal to reduce the friction of starting a business. @HillaryClinton has https://t.co/OhFAFEFsUq
The election is just weeks away. Check if you're registered to vote at https://t.co/HcMAh8ljR0, only takes a few cl… https://t.co/H1H7hAA4XM
Hillary Clinto

In [5]:
cnt = collections.Counter()
for text in train_input['tweet']:
    for word in text:
        cnt[word] += 1

print(len(cnt))

142


In [6]:
k = 0
dictionary = dict()
for text in train_input['tweet']:
    for word in text:
        if word not in dictionary:
            k += 1
            dictionary[word] = k

print(len(dictionary))
print(dictionary)

142
{'T': 1, 'h': 2, 'e': 3, ' ': 4, 'q': 5, 'u': 6, 's': 7, 't': 8, 'i': 9, 'o': 10, 'n': 11, 'l': 12, 'c': 13, ':': 14, 'W': 15, 'a': 16, 'p': 17, 'w': 18, 'm': 19, 'k': 20, 'y': 21, 'r': 22, 'f': 23, 'b': 24, '?': 25, '/': 26, '.': 27, 'X': 28, 'E': 29, 'Y': 30, '9': 31, 'O': 32, 'G': 33, 'L': 34, 'g': 35, ',': 36, 'D': 37, 'd': 38, 'x': 39, '"': 40, 'I': 41, 'U': 42, '0': 43, 'B': 44, 'j': 45, '7': 46, 'z': 47, 'F': 48, "'": 49, '\n': 50, 'M': 51, 'v': 52, 'N': 53, 'Q': 54, '3': 55, 'J': 56, '-': 57, 'A': 58, 'C': 59, '!': 60, '…': 61, 'Z': 62, 'S': 63, ')': 64, 'H': 65, '@': 66, '8': 67, 'R': 68, '1': 69, '4': 70, '#': 71, 'P': 72, 'K': 73, '5': 74, '(': 75, '6': 76, '2': 77, '$': 78, '’': 79, '“': 80, '”': 81, '—': 82, 'V': 83, 'ñ': 84, 'ó': 85, '&': 86, ';': 87, '%': 88, '_': 89, '=': 90, '‘': 91, '+': 92, '→': 93, '~': 94, '🎤': 95, '*': 96, 'í': 97, '🇺': 98, '🇸': 99, '[': 100, ']': 101, '🎓': 102, '📚': 103, '🏡': 104, '🍕': 105, '🐣': 106, 'é': 107, '®': 108, '–': 109, '🎧': 110, '|

In [7]:
# charset = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ~!@#$%^&*()_+-=[]{}\|;\',./:<>? "'
# dic = {}
# j = 0
# for i in charset:
#     j = j + 1
#     dic[i] = j;

In [8]:
def preprocess(dataset):
    ret = []
    for row in dataset:
        new_line = []
        for i in row:
            if(i in dictionary):
                new_line += [[dictionary[i]]]
                last_space = False
        ret += [new_line]
    return ret

pre_x = preprocess(train_input['tweet'])
print(len(pre_x))
print(pre_x[:3])

4743
[[[1], [2], [3], [4], [5], [6], [3], [7], [8], [9], [10], [11], [4], [9], [11], [4], [8], [2], [9], [7], [4], [3], [12], [3], [13], [8], [9], [10], [11], [14], [4], [15], [2], [10], [4], [13], [16], [11], [4], [17], [6], [8], [4], [8], [2], [3], [4], [17], [12], [16], [11], [7], [4], [9], [11], [8], [10], [4], [16], [13], [8], [9], [10], [11], [4], [8], [2], [16], [8], [4], [18], [9], [12], [12], [4], [19], [16], [20], [3], [4], [21], [10], [6], [22], [4], [12], [9], [23], [3], [4], [24], [3], [8], [8], [3], [22], [25], [4], [2], [8], [8], [17], [7], [14], [26], [26], [8], [27], [13], [10], [26], [28], [22], [3], [29], [30], [31], [32], [9], [13], [33]], [[34], [16], [7], [8], [4], [11], [9], [35], [2], [8], [36], [4], [37], [10], [11], [16], [12], [38], [4], [1], [22], [6], [19], [17], [4], [7], [16], [9], [38], [4], [11], [10], [8], [4], [17], [16], [21], [9], [11], [35], [4], [8], [16], [39], [3], [7], [4], [18], [16], [7], [4], [40], [7], [19], [16], [22], [8], [27], [40], [4]

In [9]:
y = []
for i in train_input['handle']:
    if i == "HillaryClinton":
        y.append([1, 0])
    if i == "realDonaldTrump":
        y.append([0, 1])
print(len(y))
print(y[:5])

4743
[[1, 0], [1, 0], [1, 0], [1, 0], [0, 1]]


In [10]:
def align(data):
    max_len = 0
    for row in data:
        if len(row) > max_len:
            max_len = len(row)
    ret = []
    for row in data:
        ret += [row + [[0]]*(max_len-len(row))]
    return ret, max_len

x, seq_max_len = align(pre_x)

In [11]:
import random
r_index = list(range(len(x)))
random.shuffle(r_index)
train_x = [x[i] for i in r_index[:int(len(r_index)*0.8)]]
train_y = [y[i] for i in r_index[:int(len(r_index)*0.8)]]
test_x = [x[i] for i in r_index[int(len(r_index)*0.8):]]
test_y = [y[i] for i in r_index[int(len(r_index)*0.8):]]

In [12]:
print(train_x[:2])
print(len(train_x)+len(test_x))
print(len(train_y))
print(seq_max_len)
print(train_y[:10])

[[[40], [15], [3], [4], [16], [22], [3], [4], [7], [8], [22], [10], [11], [35], [3], [22], [4], [18], [2], [3], [11], [4], [18], [3], [4], [22], [3], [7], [17], [3], [13], [8], [4], [3], [16], [13], [2], [4], [10], [8], [2], [3], [22], [36], [4], [12], [9], [7], [8], [3], [11], [4], [8], [10], [4], [3], [16], [13], [2], [4], [10], [8], [2], [3], [22], [36], [4], [16], [11], [38], [4], [16], [13], [8], [4], [18], [9], [8], [2], [4], [16], [4], [7], [3], [11], [7], [3], [4], [10], [23], [4], [13], [10], [19], [19], [10], [11], [4], [17], [6], [22], [17], [10], [7], [3], [27], [40], [4], [82], [65], [9], [12], [12], [16], [22], [21], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], [[69], [31], [46], [31], [27], [4], [2], [8], [8], [17], [7], [14], [26], [26], [8], [27], [13], [10], [26], [83], [30], [1], [35], [12], [2], [72], [21], [23], [32], [0], [0], [0], [0], [0], [0], [0]

In [19]:
n_hidden = 60
num_steps = 5000
n_classes = 2
state_size = 64
batch_size = 200

NUM_LAYERS=2

def LstmCell():
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, state_is_tuple=True)
    return lstm_cell


def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    
    def dynamicRNN(x, weights, biases):
        seqlen = length(x)
        x = tf.unstack(x, seq_max_len, 1)
#         lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
#         lstm_cell = tf.contrib.rnn.MultiRNNCell([LstmCell() for _ in range(NUM_LAYERS)])
        cell = tf.nn.rnn_cell.GRUCell(state_size)
        init_state = tf.get_variable('gruCell', [1, state_size], initializer=tf.constant_initializer(0.0))
        init_state = tf.tile(init_state, [batch_size, 1])
        outputs, final_state = tf.nn.dynamic_rnn(cell, x, sequence_length=seqlen, initial_state=init_state)
#         outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32, sequence_length=seqlen)

        # When performing dynamic calculation, we must retrieve the last
        # dynamically computed output, i.e., if a sequence length is 10, we need
        # to retrieve the 10th output.
        # However TensorFlow doesn't support advanced indexing yet, so we build
        # a custom op that for each sample in batch size, get its length and
        # get the corresponding relevant output.

        # 'outputs' is a list of output at every timestep, we pack them in a Tensor
        # and change back dimension to [batch_size, n_step, n_input]
        outputs = tf.stack(outputs)
        outputs = tf.transpose(outputs, [1, 0, 2])

        # Hack to build the indexing and retrieve the right output.
        batch_size = tf.shape(outputs)[0]
        # Start indices for each sample
        index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
        # Indexing
        outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

        # Linear activation, using outputs computed above
        return tf.matmul(outputs, weights['out']) + biases['out']
        
    x = tf.placeholder("float", [None, seq_max_len,1])
    y = tf.placeholder("float", [None, n_classes])
    inputs = x
    labels = y
    logits = dynamicRNN(x, weights, biases)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.5).minimize(loss)
    
    correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

UnboundLocalError: local variable 'batch_size' referenced before assignment

In [14]:
batch_start = 0;
train_length = len(train_x)
print(train_length)

train_x_extend = [i for i in train_x]
train_y_extend = [i for i in train_y]

3794


In [15]:
train_x_extend.extend(train_x[0:batch_size])
train_y_extend.extend(train_y[0:batch_size])

In [16]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        feed_dict = {inputs: train_x[batch_start:batch_start+batch_size], labels: train_y[batch_start:batch_start+batch_size]}
        batch_start += batch_size
        if(batch_start>=train_length):
            batch_start -=train_length
        _, l, predictions, acc = session.run([optimizer, loss, logits, accuracy],feed_dict = feed_dict)
        if (step % 100 == 0):
            print('Loss at step %d: %f acc: %f' % (step, l,acc))
    print("Testing Accuracy:", session.run(accuracy, feed_dict={inputs: test_x, labels: test_y}))

Initialized


InvalidArgumentError: Input to reshape is a tensor with 1894400 values, but the requested shape requires a multiple of 60
	 [[Node: Reshape = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](transpose, Reshape/shape)]]
	 [[Node: add_1/_321 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_57179_add_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Reshape', defined at:
  File "/home/ziyi/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ziyi/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ziyi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ziyi/anaconda3/lib/pyt